<a href="https://colab.research.google.com/github/mutherr/CS6120-PS1/blob/master/PS1_Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
import numpy as np

In [0]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

In [0]:
#Here is where you will featurize the data.
#NB: The current contents are for testing only
#This function should return: 
#  -a numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 
def createFeatures(corpus):
  from sklearn.feature_extraction.text import CountVectorizer

  texts = [entry["text"] for entry in corpus]
  genres = [entry["genre"] for entry in corpus]

  vectorizer = CountVectorizer()
  texts = vectorizer.fit_transform(texts)
  vocab = vectorizer.get_feature_names()

  return texts,genres,vocab

In [0]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  print("The model's average accuracy is %f"%avg_score)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  print("The most informative terms for comedies are: %s"%pos_indicators)
  print("The most informative terms for tragedies are: %s"%neg_indicators)

def evaluateModelL2(X,y,vocab):
  return evaluateModel(X,y,vocab,penalty="l2")

In [39]:
#Run this to read the corpus and fit the model using your featurization scheme

corpus = readShakespeare()

X,y,vocab = createFeatures(corpus)

#this call will fit a model with L1 normalization
#print("----------L1 Norm-----------")
#evaluateModel(X,y,vocab)
#this call will fit a model with L2 normalization
print("----------L2 Norm-----------")
evaluateModelL2(X,y,vocab)

----------L2 Norm-----------
  (0, 499)	1
  (0, 19258)	85
  (0, 17221)	345
  (0, 5706)	4
  (0, 2342)	97
  (0, 19433)	1
  (0, 15298)	1
  (0, 5273)	1
  (0, 12546)	1
  (0, 9540)	136
  (0, 11735)	476
  (0, 7098)	19
  (0, 17225)	708
  (0, 5396)	29
  (0, 6809)	21
  (0, 1602)	136
  (0, 3840)	40
  (0, 14635)	28
  (0, 9649)	118
  (0, 596)	58
  (0, 11777)	17
  (0, 10166)	209
  (0, 12223)	175
  (0, 6877)	1
  (0, 17930)	23
  :	:
  (25, 18126)	1
  (25, 420)	1
  (25, 5582)	1
  (25, 628)	1
  (25, 1021)	1
  (25, 9445)	1
  (25, 14575)	1
  (25, 3677)	1
  (25, 14139)	1
  (25, 4902)	1
  (25, 1944)	1
  (25, 7111)	1
  (25, 7305)	1
  (25, 15622)	1
  (25, 2550)	1
  (25, 16752)	1
  (25, 2901)	1
  (25, 14662)	1
  (25, 11618)	1
  (25, 12383)	1
  (25, 12564)	1
  (25, 19463)	1
  (25, 6282)	1
  (25, 4820)	1
  (25, 9873)	1 ['comedy', 'tragedy', 'comedy', 'comedy', 'tragedy', 'tragedy', 'tragedy', 'tragedy', 'tragedy', 'comedy', 'tragedy', 'comedy', 'comedy', 'comedy', 'comedy', 'comedy', 'tragedy', 'tragedy', 'comed